# Python Finance & Stocks

In [297]:
import datetime as dt 
import matplotlib.pyplot as plt 
from matplotlib import style #style your graphs
import pandas as pd 
import pandas_datareader.data as web #used to get stock data
from pandas.testing import assert_frame_equal #fixes warning message
import matplotlib.dates as mdates
import bs4 as bs
import pickle
import requests

## Get S&P 500 Symbols

In [298]:
def get_sp500_symbols():
    resp=requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup=bs.BeautifulSoup(resp.text,"lxml")
    table=soup.find('table',{'class':'wikitable sortable'})
    tickers=[]
    for row in table.findAll('tr')[1:]:
        ticker=row.findAll('td')[0].text
        tickers.append(ticker.replace('\n',''))
    with open("sp500symbols.pickle","wb") as f:
        pickle.dump(tickers,f)
        #print(tickers)
    return tickers

## Get S&P 500 Company Names

In [299]:
def get_sp500_names():
    resp=requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup=bs.BeautifulSoup(resp.text,"lxml")
    table=soup.find('table',{'class':'wikitable sortable'})
    company_names=[]
    for row in table.findAll('tr')[1:]:
        company=row.findAll('td')[1].text
        company_names.append(company.replace('\n',''))
    with open("sp500names.pickle","wb") as f:
        pickle.dump(company_names,f)
        #print(tickers)
    return company_names

In [300]:
sp500_symbols=get_sp500_symbols() #assign symbols to list variable
sp500_company_name=get_sp500_names() #assign company names to list variable

## Create Table of S&P 500 Companies and Symbols

In [327]:
sp500=pd.DataFrame({'Symbols':sp500_symbols,'Company Name':sp500_company_name})

In [329]:
sp500.Symbols[sp500['Symbols']=='BRK.B']='BRK-B'
sp500.Symbols[sp500['Symbols']=='BF.B']='BF-B'
sp500=sp500.set_index('Symbols')

In [322]:
style.use('ggplot') #style of graphs

## Get S&P 500 historical market data

In [323]:
start=dt.date(2019,4,17) 
end=dt.date(2020,4,17)
df=web.DataReader(list(sp500.Symbols),'yahoo',start,end)

In [398]:
latest_day=df['Adj Close'].iloc[-1].name
previous_day=df['Adj Close'].iloc[-1].name-dt.timedelta(days=1)


def get_month(latest_day,months):
    if (latest_day-pd.DateOffset(months=months)).isoweekday()==6:
        return (latest_day-pd.DateOffset(months=months))-dt.timedelta(days=1)
    elif (latest_day-pd.DateOffset(months=months)).isoweekday()==7:
        return ((latest_day-pd.DateOffset(months=months))+dt.timedelta(days=1))
    else:
        return latest_day-pd.DateOffset(months=months) 

def get_date(latest_day,days):
    if (latest_day-dt.timedelta(days=days)).isoweekday()==6:
        return (latest_day-dt.timedelta(days=days))-dt.timedelta(days=1)
    elif (latest_day-dt.timedelta(days=days)).isoweekday()==7:
        return ((latest_day-dt.timedelta(days=days))+dt.timedelta(days=1))
    else:
        return latest_day-dt.timedelta(days=days)
    
    
previous_month=get_month(latest_day,1)
previous_3_month=get_month(latest_day,3)
previous_year=get_date(latest_day,365)

## S&P 500 Companies Percent Change

In [399]:
percent_change_prev_day=pd.Series(((df['Adj Close'].loc[latest_day]-df['Open'].loc[latest_day])/df['Open'].loc[latest_day])*100,name='Day Percent Change')
percent_change_abs_prev_day=pd.Series(abs((df['Adj Close'].iloc[-1,:]-df['Open'].loc[latest_day])/df['Open'].loc[latest_day])*100,name='Day Absolute Percent Change')

percent_change_prev_week=pd.Series(((df['Adj Close'].iloc[-1,:]-df['Open'].iloc[-5,:])/df['Open'].iloc[-5,:])*100,name='Week Percent Change')
percent_change_abs_prev_week=pd.Series(abs((df['Adj Close'].iloc[-1,:]-df['Open'].iloc[-5,:])/df['Open'].iloc[-5,:])*100,name='Week Absolute Percent Change')

percent_change_prev_month=pd.Series(((df['Adj Close'].loc[latest_day]-df['Open'].loc[previous_month])/df['Open'].loc[previous_month])*100,name='Month Percent Change')
percent_change_abs_prev_month=pd.Series(abs((df['Adj Close'].iloc[-1,:]-df['Open'].loc[previous_month])/df['Open'].loc[previous_month])*100,name='Month Absolute Percent Change')

percent_change_prev_3_month=pd.Series(((df['Adj Close'].loc[latest_day]-df['Open'].loc[previous_3_month])/df['Open'].loc[previous_3_month])*100,name='3 Month Percent Change')
percent_change_abs_prev_3_month=pd.Series(abs((df['Adj Close'].iloc[-1,:]-df['Open'].loc[previous_3_month])/df['Open'].loc[previous_3_month])*100,name='3 Month Absolute Percent Change')

percent_change_prev_year=pd.Series(((df['Adj Close'].loc[latest_day]-df['Open'].loc[previous_year])/df['Open'].loc[previous_year])*100,name='Year Percent Change')
percent_change_abs_prev_year=pd.Series(abs((df['Adj Close'].iloc[-1,:]-df['Open'].loc[previous_year])/df['Open'].loc[previous_year])*100,name='Year Absolute Percent Change')

percent_change=pd.concat([percent_change_prev_day,percent_change_abs_prev_day,percent_change_prev_week,
                          percent_change_abs_prev_week,percent_change_prev_month,percent_change_abs_prev_month,
                          percent_change_prev_3_month,percent_change_abs_prev_3_month,
                          percent_change_prev_year,percent_change_abs_prev_year],axis=1)


percent_change=percent_change.join(sp500)

percent_change=percent_change[percent_change.columns[0:-1].insert(0,percent_change.columns[-1])]

In [400]:
percent_change.sort_values('Year Percent Change',ascending=False).reset_index().style.background_gradient(cmap='Blues')

## S&P 500 Top 10 Best Performing Stocks Day

In [372]:
percent_change.reset_index()[['Symbols','Company Name','Day Percent Change']].sort_values('Day Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,Day Percent Change
0,HFC,HollyFrontier Corp,14.685913
1,APA,Apache Corporation,13.959731
2,HES,Hess Corporation,13.216116
3,DVN,Devon Energy,12.713935
4,MPC,Marathon Petroleum,12.560814
5,VLO,Valero Energy,12.156006
6,CFG,Citizens Financial Group,11.817672
7,EOG,EOG Resources,11.424770
8,FANG,Diamondback Energy,11.064135
9,OXY,Occidental Petroleum,10.543392


## S&P 500 Top 10 Worst Performing Stocks Day

In [374]:
percent_change.reset_index()[['Symbols','Company Name','Day Percent Change']].sort_values('Day Percent Change',ascending=True).reset_index(drop=True).head(10).style.background_gradient(cmap='Reds')

,Symbols,Company Name,Day Percent Change
0,LYV,Live Nation Entertainment,-6.142932
1,STT,State Street Corp.,-5.600383
2,MGM,MGM Resorts International,-5.439895
3,CNC,Centene Corporation,-4.295875
4,LUV,Southwest Airlines,-4.030768
5,ANTM,Anthem,-4.010610
6,MKTX,MarketAxess,-3.961925
7,SPG,Simon Property Group Inc,-3.838248
8,UNH,United Health Group Inc.,-3.465229
9,QRVO,Qorvo,-3.230516


## S&P 500 Best Performing Stocks Month

In [375]:
percent_change.reset_index()[['Symbols','Company Name','Month Percent Change']].sort_values('Month Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,Month Percent Change
0,APA,Apache Corporation,104.086540
1,NBL,Noble Energy Inc,78.663229
2,FANG,Diamondback Energy,71.490761
3,CPRI,Capri Holdings,71.031751
4,DRI,Darden Restaurants,70.433149
5,HAS,Hasbro Inc.,69.255000
6,CMG,Chipotle Mexican Grill,66.718157
7,DXC,DXC Technology,65.016146
8,WYNN,Wynn Resorts Ltd,62.611397
9,WMB,Williams Cos.,61.428573


## S&P 500 Worst Performing Stocks Month

In [368]:
percent_change.reset_index()[['Symbols','Company Name','Month Percent Change']].sort_values('Month Percent Change',ascending=True).reset_index(drop=True).head(10).style.background_gradient(cmap='Reds')

,Symbols,Company Name,Month Percent Change
0,AAL,American Airlines Group,-18.750001
1,DAL,Delta Air Lines Inc.,-18.529709
2,NWSA,News Corp. Class A,-15.799434
3,REG,Regency Centers Corporation,-14.317374
4,BEN,Franklin Resources,-13.315651
5,LUV,Southwest Airlines,-12.828391
6,FRT,Federal Realty Investment Trust,-12.239434
7,PBCT,People's United Financial,-12.165265
8,KIM,Kimco Realty,-12.118223
9,HRB,H&R Block,-11.358031


## S&P 500 Best Performing Stocks 3 Month

In [391]:
percent_change.reset_index()[['Symbols','Company Name','3 Month Percent Change']].sort_values('3 Month Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,3 Month Percent Change
0,REGN,Regeneron Pharmaceuticals,47.316749
1,NEM,Newmont Corporation,35.104925
2,GILD,Gilead Sciences,32.267713
3,CTXS,Citrix Systems,28.390953
4,AMZN,Amazon.com Inc.,25.935234
5,SBAC,SBA Communications,25.580371
6,NFLX,Netflix Inc.,24.035188
7,INCY,Incyte,23.487276
8,CLX,The Clorox Company,23.068120
9,COG,Cabot Oil & Gas,20.598390


## S&P 500 Worst Performing Stocks 3 Month

In [392]:
percent_change.reset_index()[['Symbols','Company Name','3 Month Percent Change']].sort_values('3 Month Percent Change',ascending=True).reset_index(drop=True).head(10).style.background_gradient(cmap='Reds')

,Symbols,Company Name,3 Month Percent Change
0,IR,Ingersoll Rand,-79.882945
1,NCLH,Norwegian Cruise Line Holdings,-78.963466
2,CCL,Carnival Corp.,-75.587949
3,APA,Apache Corporation,-74.334948
4,RCL,Royal Caribbean Cruises Ltd,-71.992510
5,OXY,Occidental Petroleum,-70.844920
6,NBL,Noble Energy Inc,-70.563322
7,HAL,Halliburton Co.,-68.416667
8,UAL,United Airlines Holdings,-67.616928
9,MRO,Marathon Oil Corp.,-67.440120


## S&P 500 Top 10 Best Performing Stocks Year

In [369]:
percent_change.reset_index()[['Symbols','Company Name','Year Percent Change']].sort_values('Year Percent Change',ascending=False).reset_index(drop=True).head(10).style.background_gradient(cmap='Greens')

,Symbols,Company Name,Year Percent Change
0,AMD,Advanced Micro Devices Inc,105.072455
1,NEM,Newmont Corporation,73.440694
2,RMD,ResMed,68.479043
3,REGN,Regeneron Pharmaceuticals,64.200343
4,MKTX,MarketAxess,64.088262
5,HUM,Humana Inc.,59.804089
6,SBAC,SBA Communications,59.383379
7,VRTX,Vertex Pharmaceuticals Inc,59.131549
8,NVDA,Nvidia Corporation,55.854133
9,EQIX,Equinix,54.375012


## S&P 500 Top 10 Worst Performing Stocks Year

In [370]:
percent_change.reset_index()[['Symbols','Company Name','Year Percent Change']].sort_values('Year Percent Change',ascending=True).reset_index(drop=True).head(10).style.background_gradient(cmap='Reds')

,Symbols,Company Name,Year Percent Change
0,NCLH,Norwegian Cruise Line Holdings,-78.566481
1,OXY,Occidental Petroleum,-78.244214
2,CCL,Carnival Corp.,-77.017383
3,ADS,Alliance Data Systems,-76.935728
4,APA,Apache Corporation,-76.866486
5,DXC,DXC Technology,-76.466073
6,IR,Ingersoll Rand,-76.238590
7,HAL,Halliburton Co.,-76.088329
8,MRO,Marathon Oil Corp.,-75.479143
9,NBL,Noble Energy Inc,-74.514119
